<a href="https://colab.research.google.com/github/johnnybigoode/practical-time-series/blob/main/TimeSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preface

Starts with examples of things that form time series, like weather, stock markets and heartbeats.

Reminds us of the future of IoT and the amount of sensors and gadgets available for high-quality time series data

Expects a introductory statistics background

There are at least two youtube 3h long scipy talks with the author on the subject

https://www.youtube.com/watch?v=JNfxr4BQrLk

https://www.youtube.com/watch?v=v5ijNXvlC5A&t=1s

book errata
https://www.oreilly.com/catalog/errata.csp?isbn=0636920187714

github repo
https://github.com/PracticalTimeSeriesAnalysis/BookRepo

# Chapter 1 Time Series: Overview and Quick History

Time series: Data in choronological order

Time series analysis: Diagnose the past and predict the future

Medical field examples:

electrocardiograms - records electrig signals passing thru the heart
electroencephalogram - measures eletrical impulses in the brain

examples of time series in weather:

barometer - taking meassurements of the state of the atmosphere at regular invervals

examples of time series in bussiness:

forecasting demand, estimaging future raw materials prices, hedging on manufacturing costs

examples of modern machine learning related to time-series:

computer security, anomaly detection as a method for getting hackers and invader dynamic time warping, a way of checking similarity of time series
recursive neural networks show usefulness for extracting patterns in corrupted data

# Chapter 2 All About The Data

problems related to preprocessing time serries data

-finding time series online
Kaggle!
UCI Machine Learning Repository
UEA and UCR Time Series Classification Repository
NOAA National Centers for Enviromental Information
Bureau of Labor Statiscs
Centers for Disease Control and Prevention
Federal Reserve Bank of St Louis
CompEngine
Mcomop and M4comp2018 R Packages
CRAN Repository

*time series can be related to anything related that is sorted in time, such as the spectrum of a wine, with the wavelenghts evenly spaced on the x-axis

*does this mean that using Fourier we can transform things in the frequency domain, to time domain and its somehow related?

*data usually must be assembled from a collection of data

Lookahead: time series information that was unknown when designing, traning or evalutating a model. This can and will screw things up. There's no test for it, something one must be vigilant and thoughtful.

When counting information, remember to put back the data you removed, aquelas coisas relacionadas a calular incluindo os extremos



In [1]:
#table 2-1 https://raw.githubusercontent.com/johnnybigoode/BookRepo/master/Ch02/data/year_joined.csv
#table 2-2 https://raw.githubusercontent.com/johnnybigoode/BookRepo/master/Ch02/data/emails.csv
#table 2-3 https://raw.githubusercontent.com/johnnybigoode/BookRepo/master/Ch02/data/donations.csv

import pandas as pd

YearJoined = pd.read_csv(
    "https://raw.githubusercontent.com/johnnybigoode/BookRepo/master/Ch02/data/year_joined.csv") 
print(YearJoined.head())

Emails = pd.read_csv("https://raw.githubusercontent.com/johnnybigoode/BookRepo/master/Ch02/data/emails.csv")
print(Emails.head())

Donations = pd.read_csv("https://raw.githubusercontent.com/johnnybigoode/BookRepo/master/Ch02/data/donations.csv")
print(Donations.head())

   user userStats  yearJoined
0     0    silver        2014
1     1    silver        2015
2     2    silver        2016
3     3    bronze        2018
4     4    silver        2018
   emailsOpened  user                 week
0           3.0   1.0  2015-06-29 00:00:00
1           2.0   1.0  2015-07-13 00:00:00
2           2.0   1.0  2015-07-20 00:00:00
3           3.0   1.0  2015-07-27 00:00:00
4           1.0   1.0  2015-08-03 00:00:00
   amount            timestamp  user
0    25.0  2017-11-12 11:13:44   0.0
1    50.0  2015-08-25 19:01:45   0.0
2    25.0  2015-03-26 12:03:47   0.0
3    50.0  2016-07-06 12:24:55   0.0
4    50.0  2016-05-11 18:13:04   1.0


In [7]:
#groupby example on dataframe
YearJoined.groupby('userStats').count()

,user,yearJoined
userStats,,
bronze,486,486
gold,95,95
inactive,96,96
silver,323,323


In [8]:
Emails[Emails.emailsOpened < 1]

,emailsOpened,user,week


In [25]:
Emails[Emails.user == 998].head()

,emailsOpened,user,week
25464,1.0,998.0,2017-12-04 00:00:00
25465,3.0,998.0,2017-12-11 00:00:00
25466,3.0,998.0,2017-12-18 00:00:00
25467,3.0,998.0,2018-01-01 00:00:00
25468,3.0,998.0,2018-01-08 00:00:00


In [2]:
#how many weekly observations betewenn first and last members
#convert week to timestamp
Emails.week = pd.to_datetime(Emails.week)

maxdate = max(Emails[Emails.user == 998].week)
mindate = min(Emails[Emails.user == 998].week)

print((maxdate - mindate).days/7)

25.0


In [34]:
#how many weeks of data
#notice we should have 26
Emails[Emails.user == 998].shape

(24, 3)

# Why 26 Rows?

When doing time subtraction you should ask: should i add 1 to account for the offset at the end?

Example

I got info for 7, 14, 21 and 28
How many points do I have in total?

28-7 / 7 = 21/7 = 3

But we obviously have 4 points, so we need to add 1 to account for the subtracted start date

Then again, 28/7 = 4, why did was it subtracted in the first place?

In [11]:
#let's fill all missing weeks
#multiindex - a cartesian product to create combinations of weeks and members
complete_idx = pd.MultiIndex.from_product((set(Emails.week),set(Emails.user)))
print(complete_idx)

#using this new index, let's fill missing values with 0
all_email = Emails.set_index(['week', 'user']).reindex(complete_idx, fill_value = 0).reset_index()
#changing columns names so it matches what is being used in the book  I guess
all_email.columns = ['week', 'member', 'EmailsOpened']


MultiIndex([('2017-07-03',   1.0),
            ('2017-07-03',   3.0),
            ('2017-07-03',   5.0),
            ('2017-07-03',   6.0),
            ('2017-07-03',   9.0),
            ('2017-07-03',  10.0),
            ('2017-07-03',  14.0),
            ('2017-07-03',  16.0),
            ('2017-07-03',  20.0),
            ('2017-07-03',  21.0),
            ...
            ('2018-02-19', 973.0),
            ('2018-02-19', 977.0),
            ('2018-02-19', 982.0),
            ('2018-02-19', 984.0),
            ('2018-02-19', 987.0),
            ('2018-02-19', 991.0),
            ('2018-02-19', 992.0),
            ('2018-02-19', 993.0),
            ('2018-02-19', 995.0),
            ('2018-02-19', 998.0)],
           length=93247)


In [13]:
all_email[all_email.member == 998].sort_values('week')
#we can see the empty weeks since the first register 

,week,member,EmailsOpened
44736,2015-02-09,998.0,0.0
81388,2015-02-16,998.0,0.0
80849,2015-02-23,998.0,0.0
87317,2015-03-02,998.0,0.0
26949,2015-03-09,998.0,0.0
...,...,...,...
39885,2018-04-30,998.0,3.0
50126,2018-05-07,998.0,3.0
12396,2018-05-14,998.0,3.0
77076,2018-05-21,998.0,3.0


In [14]:
#the zeroes at the start would mean when the member joined the organization
#the zeros at the end would mean when he left the organization
#let's trim it
cutoff_dates = Emails.groupby('user').week.agg(['min', 'max']).reset_index()
print(cutoff_dates.head())
cutoff_dates = cutoff_dates.reset_index()

   user        min        max
0   1.0 2015-06-29 2018-05-28
1   3.0 2018-03-05 2018-04-23
2   5.0 2017-06-05 2018-05-28
3   6.0 2016-12-05 2018-05-28
4   9.0 2016-07-18 2018-05-28


In [18]:
#this piece of code will generate a bunch of usererrors 
#for more infor https://stackoverflow.com/questions/41710789/boolean-series-key-will-be-reindexed-to-match-dataframe-index
#choose NOT to fix it to use the book's code
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

#dropping the zeroes
for _, row in cutoff_dates.iterrows():
  member = row['user']
  start_date = row['min']
  end_date = row['max']
  
  all_email.drop(
      all_email[all_email.member == member][all_email.week < start_date].index, 
      inplace=True)
  
  all_email.drop(
      all_email[all_email.member == member][all_email.week > end_date].index, 
      inplace=True)

#enables userwarning
warnings.simplefilter(action='error', category=UserWarning)

In [22]:
#the initial zeroes @ 2015 are gone
print(all_email[all_email.member == 998].sort_values('week').head(10))

            week  member  EmailsOpened
49587 2017-12-04   998.0           1.0
77615 2017-12-11   998.0           3.0
29644 2017-12-18   998.0           3.0
85161 2017-12-25   998.0           0.0
1616  2018-01-01   998.0           3.0
40963 2018-01-08   998.0           3.0
87856 2018-01-15   998.0           2.0
4850  2018-01-22   998.0           3.0
53899 2018-01-29   998.0           2.0
40424 2018-02-05   998.0           3.0


In [ ]:
#like we did with Emails, convert the date that is a string into timestamp
#don't run this twice, the timestamp column is gone after set index
Donations.timestamp = pd.to_datetime(Donations.timestamp)
Donations.set_index('timestamp', inplace = True)

In [32]:
#https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html
#functional programming apply anon function / https://en.wikipedia.org/wiki/Apply

#resample is a pandas function @ https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.resample.html#pandas.Series.resample
#notice that we apply the resample to the .amount series
#the "W-MON" is the rule, more information @ https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#resampling
agg_don = Donations.groupby('user').apply(
  lambda df: df.amount.resample("W-MON").sum().dropna()
)

#results look weird tho
print(agg_don.head(15))

user  timestamp 
0.0   2015-03-30    25.0
      2015-04-06     0.0
      2015-04-13     0.0
      2015-04-20     0.0
      2015-04-27     0.0
      2015-05-04     0.0
      2015-05-11     0.0
      2015-05-18     0.0
      2015-05-25     0.0
      2015-06-01     0.0
      2015-06-08     0.0
      2015-06-15     0.0
      2015-06-22     0.0
      2015-06-29     0.0
      2015-07-06     0.0
Name: amount, dtype: float64


In [49]:
#we should be able to join the agg_don to the all_emails things
merged_df = pd.DataFrame()
for member, member_email in all_email.groupby('member'):
  #fix?
  member_donations = agg_don[member]
  #it's already timestamp
  #member_donations.set_index('timestamp', inplace=True)
  member_email.set_index('week', inplace=True)
  member_email = all_email[all_email.member == member]
  member_email.sort_values('week').set_index('week')

  df = pd.merge(
      member_email, 
      member_donations, 
      how = 'left', 
      left_index = True, 
      right_index = True)
  
  df.fillna(0)

  #why member_x and not member?
  df['member'] = member
  print(df)
  merged_df = merged_df.append(df)

            week  member  EmailsOpened  amount
0     2017-07-03     1.0           3.0     NaN
539   2017-06-26     1.0           3.0     NaN
1078  2018-01-01     1.0           3.0     NaN
1617  2015-08-03     1.0           1.0     NaN
2156  2017-05-01     1.0           3.0     NaN
...          ...     ...           ...     ...
90552 2017-10-09     1.0           3.0     NaN
91091 2016-03-14     1.0           3.0     NaN
91630 2017-10-30     1.0           3.0     NaN
92169 2015-07-13     1.0           2.0     NaN
92708 2018-02-19     1.0           3.0     NaN

[153 rows x 4 columns]


KeyError: ignored

RECAPING SOME TECHNIQUES
from worked example pg 27-35

1. recalibrate data resolution. data might come daily, but we only need weekly.

2. avoid lookahead by NOT using data for timestamps that produces data availability???

3. record all relevant time periods even if nothing happened. a zero count is as important as any other

4. avoid lookahead by NOT using data for timestamps that produce information we shiouldn't know yet about

Timestamps are specially tricky to work with due to their complexity. It is important to understand what a ts means. Is it when the event happened? Or was it when it was recorded into a table? Maybe it was when the user logged into the system? Or when the data migrated tables. If you can, reading the code that generated the data might help, even better would be talking to those who wrote the code, and in a perfect world, documentation that correctly reflects code. 

In most cases, we'll have to do some imperical research.

Analyse a single user's tss and then expand to other users. Always consider that the ts might be the user's localtime, meaning, you can notice usage behavior to be directly correlated to choronology.

You can also subtract the previous entry ts from the current and look for zeroes. Several entries on the same ts indicate a server behavior.

Psychological Time Discounting / Psychological distance. People are more optimistic when estimating things in the distant future

Cleaning data basics:

-missing data
	imputation, fill in missing data based on the whole database
	iterpolation, use near points to predict missing values 
	deletion of affected time periods, ignore time periods we don't have data

R data.table has a rolling join that seems to be amazing the joining on timestamps and a zoo package that has a bunch of timeseries related functions

foward fill makes more sense even if fancier methods are possible, think about what the missing data could be

take notice when using moving averages and using mean deviations when inerpolating because lookaheads

-upsampling and downsampling
	select output every nth element
	to focus on seasons, pick a single month every year, or a single day every week
	in irregular time series, rolling join
	
-smoothing
	why tho?
	data preparation
	feature generation
	prediction
	visualization
	
it's important to check outcomes affected by smoothing

using exponential smoothing (and averages on this matter) gives more weight to recent data vs old data
>check more resources on page 69

pandas function ewma for exponential smoothing

smoothing is so comon that you should use it as a easy null model when testing against fancier methods

starting smoothing calculations may be problematic, if you think about two points, a exponential operation would consider the first point to be the sum of knowledge of what came before

>Seasonal Data

to see seasonal data, a different plot might help.

pg62 shows a single line of R to decompose components: seasonal, trend, remainder

documentation available at

https://www.rdocumentation.org/packages/stats/versions/3.6.2/topics/stl

TIMEZONES: everyone hates them
pickling timezone objects is problematic 
why? bloggers mentioned it

PREVENTING LOOKAHEAD

-when smoothing data, experiment
-build everything on small data and introduce random spot checks
-for each kind of data, check what the timestamps relate to
-use rolling testing or cross validation, RANDOMIZING TRAINING VERSUS TESTING DATA DOES NOT WORK WITH TIME SERIES
-intentionally introduce lookahead, if you know a model's accuracy with lookahead, it's easier to set a maximum for the model ur building
-add features slowly, any jump in acc should be explaned by lookahead

>Review of functional data
analysis / https://perma.cc/3DNT-J9EZ